In [1]:
from dotenv import load_dotenv
import os

load_dotenv()



True

In [3]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")


In [ ]:
from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch
# from db.config import rev_embeddings_collection

llm = init_chat_model(model="llama3-8b-8192", model_provider="groq")
embeddings = HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

embed = embeddings.embed_query("text")
print(len(embed))

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.